In [169]:
import pandas as pd
import numpy as np

df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Practice/ANN/drug200.csv')
df.head()

,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,F,HIGH,HIGH,25.355,DrugY
1,47,M,LOW,HIGH,13.093,drugC
2,47,M,LOW,HIGH,10.114,drugC
3,28,F,NORMAL,HIGH,7.798,drugX
4,61,F,LOW,HIGH,18.043,DrugY


In [170]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Age          200 non-null    int64  
 1   Sex          200 non-null    object 
 2   BP           200 non-null    object 
 3   Cholesterol  200 non-null    object 
 4   Na_to_K      200 non-null    float64
 5   Drug         200 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 9.5+ KB


In [171]:
df.describe()

,Age,Na_to_K
count,200.000000,200.000000
mean,44.315000,16.084485
std,16.544315,7.223956
min,15.000000,6.269000
25%,31.000000,10.445500
50%,45.000000,13.936500
75%,58.000000,19.380000
max,74.000000,38.247000


In [178]:
df.columns=['age','sex','bp','choles','na_k_level','drug']
df.head()

,age,sex,bp,choles,na_k_level,drug
0,23,0,HIGH,HIGH,25.355,DrugY
1,47,1,LOW,HIGH,13.093,drugC
2,47,1,LOW,HIGH,10.114,drugC
3,28,0,NORMAL,HIGH,7.798,drugX
4,61,0,LOW,HIGH,18.043,DrugY


In [179]:
df.sex.value_counts()

1    104
0     96
Name: sex, dtype: int64

In [180]:
df.bp.value_counts()

HIGH      77
LOW       64
NORMAL    59
Name: bp, dtype: int64

In [181]:
df.drug.value_counts()

DrugY    91
drugX    54
drugA    23
drugC    16
drugB    16
Name: drug, dtype: int64

In [182]:
# Encoding theese categorical variables

In [183]:
df['sex']=pd.get_dummies(df['sex'],drop_first=True)
df.head()

,age,sex,bp,choles,na_k_level,drug
0,23,0,HIGH,HIGH,25.355,DrugY
1,47,1,LOW,HIGH,13.093,drugC
2,47,1,LOW,HIGH,10.114,drugC
3,28,0,NORMAL,HIGH,7.798,drugX
4,61,0,LOW,HIGH,18.043,DrugY


In [184]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

le=LabelEncoder()
df['bp']=le.fit_transform(df['bp'])
df.head() #high=0, low=1, normal=2

,age,sex,bp,choles,na_k_level,drug
0,23,0,0,HIGH,25.355,DrugY
1,47,1,1,HIGH,13.093,drugC
2,47,1,1,HIGH,10.114,drugC
3,28,0,2,HIGH,7.798,drugX
4,61,0,1,HIGH,18.043,DrugY


In [185]:
le2=LabelEncoder()
df['choles']=le2.fit_transform(df['choles'])
df.head() #high=0, low=1, normal=2

,age,sex,bp,choles,na_k_level,drug
0,23,0,0,0,25.355,DrugY
1,47,1,1,0,13.093,drugC
2,47,1,1,0,10.114,drugC
3,28,0,2,0,7.798,drugX
4,61,0,1,0,18.043,DrugY


In [186]:
le3=LabelEncoder()
df['drug']=le3.fit_transform(df['drug'])
df.head() #high=0, low=1, normal=2

,age,sex,bp,choles,na_k_level,drug
0,23,0,0,0,25.355,0
1,47,1,1,0,13.093,3
2,47,1,1,0,10.114,3
3,28,0,2,0,7.798,4
4,61,0,1,0,18.043,0


In [187]:
X=df.iloc[:,:-1].values
y=df.iloc[:,-1].values

In [188]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.2,random_state=3)

In [190]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()

X_train_scaled=sc.fit_transform(X_train)
X_test_scaled=sc.transform(X_test)

In [191]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation

In [219]:
LAYERS = [Dense(20,input_dim=5, activation=tf.keras.activations.relu, name='io_layer1'),
          Dense(20, activation=tf.keras.activations.relu,name='hidden_layer1'),
          Dense(20, activation=tf.keras.activations.relu,name='hidden_layer2'),
          Dense(5, activation=tf.keras.activations.softmax, name='op_layer')]

OPTIMIZER=tf.keras.optimizers.Adam(1e-3)
LOSS = tf.keras.losses.sparse_categorical_crossentropy

In [220]:
y_train
pd.Series(y_train).unique()

array([1, 0, 3, 4, 2])

In [221]:
model=Sequential(LAYERS)
model.compile(optimizer=OPTIMIZER, loss=LOSS, metrics=['accuracy'])
model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 io_layer1 (Dense)           (None, 20)                120       
                                                                 
 hidden_layer1 (Dense)       (None, 20)                420       
                                                                 
 hidden_layer2 (Dense)       (None, 20)                420       
                                                                 
 op_layer (Dense)            (None, 5)                 105       
                                                                 
Total params: 1,065
Trainable params: 1,065
Non-trainable params: 0
_________________________________________________________________


In [222]:
model.fit(X_train_scaled, y_train, epochs=50,verbose=2)

Epoch 1/50
5/5 - 0s - loss: 1.6105 - accuracy: 0.3375 - 465ms/epoch - 93ms/step
Epoch 2/50
5/5 - 0s - loss: 1.5479 - accuracy: 0.3750 - 16ms/epoch - 3ms/step
Epoch 3/50
5/5 - 0s - loss: 1.4885 - accuracy: 0.4375 - 21ms/epoch - 4ms/step
Epoch 4/50
5/5 - 0s - loss: 1.4370 - accuracy: 0.4563 - 17ms/epoch - 3ms/step
Epoch 5/50
5/5 - 0s - loss: 1.3906 - accuracy: 0.4688 - 19ms/epoch - 4ms/step
Epoch 6/50
5/5 - 0s - loss: 1.3453 - accuracy: 0.5063 - 19ms/epoch - 4ms/step
Epoch 7/50
5/5 - 0s - loss: 1.3010 - accuracy: 0.5375 - 14ms/epoch - 3ms/step
Epoch 8/50
5/5 - 0s - loss: 1.2571 - accuracy: 0.5625 - 16ms/epoch - 3ms/step
Epoch 9/50
5/5 - 0s - loss: 1.2144 - accuracy: 0.5875 - 17ms/epoch - 3ms/step
Epoch 10/50
5/5 - 0s - loss: 1.1736 - accuracy: 0.6000 - 14ms/epoch - 3ms/step
Epoch 11/50
5/5 - 0s - loss: 1.1332 - accuracy: 0.6062 - 14ms/epoch - 3ms/step
Epoch 12/50
5/5 - 0s - loss: 1.0920 - accuracy: 0.6062 - 15ms/epoch - 3ms/step
Epoch 13/50
5/5 - 0s - loss: 1.0513 - accuracy: 0.6375 - 14

In [232]:
model.predict(X_test)

2/2 [==============================] - 0s 12ms/step


array([[9.99999940e-01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [9.99999940e-01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [9.99999940e-01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [9.99999940e-01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.78013190e-36],
       [9.99999940e-01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        3.53717959e-35],
       [9.99999940e-01, 0.00000000e+00, 5.28366290e-37, 1.51226255e-33,
        3.41356397e-29],
       [9.99999940e-01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [9.99999940e-01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        8.92767207e-37],
       [9.99999940e-01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [9.99999940e-01, 0.00000000e+00, 0.00000000e+00, 4.83777197e-37,
        2.65071508e-32],
       [9.99999940e-01, 0.0000

In [229]:
y_pred=np.argmax(model.predict(X_test), axis=-1)

2/2 [==============================] - 0s 8ms/step
